In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}"

In [ ]:
program = "coloring"
# program = "dijkstra"
graph_name = "star_graph_n7"
# graph_name = "implicit_graph_n8"
sched = 0
no_simulations = 500000
me = False
fault_interval = 1

selected_nodes = [0, 3, 6]

filenames = [
    get_filename(
        graph_name,
        sched,
        SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
        "",
        no_simulations,
        me,
        fault_interval,
    )
]
filenames.extend(
    [
        get_filename(
            graph_name,
            sched,
            SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE,
            arg,
            no_simulations,
            me,
            fault_interval,
        )
        for arg in selected_nodes
    ]
)
# filenames.extend(
#     [
#         get_filename(
#             graph_name,
#             sched,
#             SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG,
#             arg,
#             no_simulations,
#             me,
#             fault_interval,
#         )
#         for arg in selected_nodes
#     ]
# )

In [5]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [6]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(36)

In [7]:
bins = np.linspace(0, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [8]:
bins

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36.])

In [9]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([    0,  4949, 18155, 35260, 46618, 53351, 54528, 52114, 46993,
       40593, 34485, 28563, 23169, 17707, 13325,  9590,  6870,  4736,
        3120,  2061,  1380,   868,   591,   346,   241,   156,    95,
          57,    31,    19,    13,     7,     3,     1,     3,     2])

In [10]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [11]:
hist_df[0]

,Steps,Count
0,0.0,0
1,1.0,4949
2,2.0,18155
3,3.0,35260
4,4.0,46618
5,5.0,53351
6,6.0,54528
7,7.0,52114
8,8.0,46993
9,9.0,40593


In [12]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count0,Count1,Count2,Count3
0,0.0,0,0,0,0
1,1.0,4949,4931,5279,2779
2,2.0,18155,19362,17465,19399
3,3.0,35260,35570,35405,35493
4,4.0,46618,46279,46240,46618
5,5.0,53351,52888,53169,53267
6,6.0,54528,54335,54491,54876
7,7.0,52114,51530,52286,52224
8,8.0,46993,47080,46940,47046
9,9.0,40593,40673,41109,40905


In [13]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count0,Count1,Count2,Count3
Steps,,,,
0.0,0,0,0,0
1.0,4949,4931,5279,2779
2.0,18155,19362,17465,19399
3.0,35260,35570,35405,35493
4.0,46618,46279,46240,46618
5.0,53351,52888,53169,53267
6.0,54528,54335,54491,54876
7.0,52114,51530,52286,52224
8.0,46993,47080,46940,47046


In [14]:
# counts, bin_edges = np.histogram(df['Steps'], bins=30)

# # Compute bin centers from bin edges
# bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
# hist_df = pd.DataFrame({
#     'Steps': bin_centers,
#     'Count': counts
# })
# hist_df

In [15]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [16]:
def get_filename():
    return f"{graph_name}__{sched}__{no_simulations:,}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}"

In [17]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, marker='o', linewidth=3)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 1:
        line_style = "solid"
    else:
        line_style = 'dashed'
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

labels = ['Random Fault']
labels.extend([f'Controlled at node {n}' for n in selected_nodes])
custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linewidth=1,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
plt.savefig(
        os.path.join(
            plots_dir,
            program,
            f"{get_filename()}.png",
        ),
        bbox_inches="tight",
    )
plt.close()